In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

df = df.loc[df['type']!=0]
df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
df['aid2'] = df.aid
print( df.shape )
df.head()

(635491, 7)


,session,aid,ts,type,hour,day,aid2
12,12899781,199008,1661781409,1,50209,395809,199008
21,12899782,1494780,1661724244,1,79444,338644,1494780
28,12899782,413962,1661765608,1,34408,380008,413962
33,12899782,779477,1661766162,1,34962,380562,779477
35,12899782,562753,1661766178,1,34978,380578,562753


In [3]:
user_features = df.groupby('session').agg({'type':'mean','aid':'count','aid2':'nunique','hour':'mean','day':'mean'})
user_features.columns = ['buy_ratio2','count_item2','unique_item2','hour_mean2','day_mean2']

In [4]:
user_features.head()

,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2
session,,,,,
12899781,1.000000,1,1,50209.000000,395809.000000
12899782,1.333333,24,20,65109.875000,407109.875000
12899786,1.000000,1,1,79217.000000,338417.000000
12899787,1.000000,3,1,79232.000000,338432.000000
12899790,1.333333,3,2,79284.333333,338484.333333


In [5]:
user_features2 = df.groupby('session').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
user_features2.columns = ['buy_ratio_std2','hour_std2','day_std2']

f32 = ['buy_ratio_std2','hour_std2','day_std2']
for c in f32: user_features2[c] = user_features2[c].astype('float32')

In [6]:
user_features2.head()

,buy_ratio_std2,hour_std2,day_std2
session,,,
14114962,-1.0,-1.000000,-1.000000
14220546,0.0,325.333099,325.333099
13115870,0.0,12.727922,12.727922
14306092,0.0,16.970562,16.970562
13534194,-1.0,-1.000000,-1.000000


In [7]:
user_features['repeat2'] = user_features.count_item2 / user_features.unique_item2

f32 = ['buy_ratio2','hour_mean2','day_mean2','repeat2']
for c in f32: user_features[c] = user_features[c].astype('float32')
    
i32 = ['count_item2','unique_item2']
for c in i32: user_features[c] = user_features[c].astype('int32')

In [8]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [9]:
user_features.columns = [x.replace('2','10') for x in user_features.columns]

In [10]:
user_features.head()

,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
session,,,,,,,,,
12899781,1.000000,1,1,50209.000000,395809.00000,1.0,-1.000000,-1.000000,-1.000000
12899782,1.333333,24,20,65109.875000,407109.87500,1.2,0.481543,13592.392578,19700.259766
12899786,1.000000,1,1,79217.000000,338417.00000,1.0,-1.000000,-1.000000,-1.000000
12899787,1.000000,3,1,79232.000000,338432.00000,3.0,0.000000,8.888194,8.888194
12899790,1.333333,3,2,79284.335938,338484.34375,1.5,0.577350,128.165253,128.165253


In [11]:
user_features.dtypes

buy_ratio10        float32
count_item10         int32
unique_item10        int32
hour_mean10        float32
day_mean10         float32
repeat10           float32
buy_ratio_std10    float32
hour_std10         float32
day_std10          float32
dtype: object

In [12]:
user_features.to_parquet('../../data/item_user_features/user10_LB.pqt')